In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random

In [88]:
class TicTacToe:
    def __init__(self):
        self.board = [[" " for _ in range(4)] for _ in range(4)]
        self.current_player = -1


    def check_draw(self):
        for row in self.board:
            if " " in row:
                return False
        return True

    def print_board(self):
        # Prints a GUI-like representation of the board
        print("┌───┬───┬───┬───┐")
        for i, row in enumerate(self.board):
            print("│ " + " │ ".join(row) + " │")
            if i < 3:
                print("├───┼───┼───┼───┤")
        print("└───┴───┴───┴───┘")

    def check_winner(self, player):
        for row in self.board:
            if all([cell == player for cell in row]):
                return True
        for col in range(4):
            if all([self.board[row][col] == player for row in range(4)]):
                return True
        if all([self.board[i][i] == player for i in range(4)]) or all(
            [self.board[i][3 - i] == player for i in range(4)]
        ):
            return True
        return False

    def step(self, state):
        row = int(state / 4)
        col = int(state % 4)

        current_player_symbol = " "
        if self.current_player == -1:
            current_player_symbol = "X"
        else:
            current_player_symbol = "O"

        if self.board[row][col] == " ":
            self.board[row][col] = current_player_symbol

        if self.check_winner("O"):
            return self.board, self.current_player, True, -1
        elif self.check_winner("X"):
            return self.board, self.current_player, True, 1
        elif self.check_draw():
            return self.board, self.current_player, True, 0

        self.current_player *= -1

        return self.board, self.current_player, False, 0

In [102]:
import pickle as pkl
with open("Q_strategys.pkl", "rb") as f:
    strategies = pkl.load(f)

# Initialized as a random policy for player 1

def policy_player1(board):

    possible_actions = []

    for i in range(4):
        for j in range(4):
            if board[i][j] == " ":
                possible_actions.append(i*4 + j)


    return random.choice(possible_actions)
    board = np.array(board).flatten()
    player = "X" if sum(board == "X") == sum(board == "O") else "O"
    strategy = strategies[f"{player}_strategy"]
    
    s_key = "".join(board)
    if s_key in strategy:
        q_values = strategy[s_key]
    else:
        q_values = np.zeros(16).astype(float)
    
    available_actions = np.where(board == " ")[0]
    legal_q_vals = q_values[available_actions]
    max_q = legal_q_vals.max()
    idxs = np.where(legal_q_vals == max_q)[0]
    return available_actions[np.random.choice(idxs)]


# Initialized as a random policy for player 2
def policy_player2(board):

    possible_actions = []
    board = np.array(board).flatten()
    player = "X" if sum(board == "X") == sum(board == "O") else "O"
    
    strategy = strategies[f"{player}_strategy"]
    s_key = "".join(board)
    if s_key in strategy:
        q_values = strategy[s_key]
    else:
        q_values = np.zeros(16).astype(float)
    
    available_actions = np.where(board == " ")[0]
    legal_q_vals = q_values[available_actions]
    max_q = legal_q_vals.max()
    idxs = np.where(legal_q_vals == max_q)[0]
    return available_actions[np.random.choice(idxs)]
    


    #return random.choice(possible_actions)

In [103]:
def play_one_game(policy_player1, policy_player2):
    tictactoe = TicTacToe()


    terminated = 0
    board = [[" " for _ in range(4)] for _ in range(4)]

    for i in range(8):
        for turn in [-1, 1]:
            action = 0
            if turn == -1:
                action = policy_player1(board)
            else:
                action = policy_player2(board)

            board, player, terminated, reward = tictactoe.step(action)

            # Uncomment this if you want to see the board
            #tictactoe.print_board()

            if terminated:
                break

    return -1*reward # This is the player who won


In [104]:
from tqdm import tqdm
def run_alternating_games(games=10):
    results = []
    for i in tqdm(range(games)):
        for j in range(2):
            if j==0:
                winner = play_one_game(policy_player1, policy_player2)

                match winner:
                    case -1:
                        results.append(1)
                    case 1:
                        results.append(2)
                    case 0:
                        results.append(0)

            if j==1:
                winner = play_one_game(policy_player2, policy_player1)

                match winner:
                    case -1:
                        results.append(2)
                    case 1:
                        results.append(1)
                    case 0:
                        results.append(0)


    return results

In [105]:
results = run_alternating_games(1000)
print("Draws: ", results.count(0))
print("Player 1 Wins:", results.count(1))
print("Player 2 Wins:", results.count(2))

100%|██████████| 1000/1000 [00:02<00:00, 467.66it/s]

Draws:  268
Player 1 Wins: 199
Player 2 Wins: 1533


I have created two functions that randomly select any action from the available actions from the board. Your team will have to create such a function that outputs the optimal action given a particular board state. This a similar kind of code I will be using on competition day when your function will play against an opponent's functions for perhaps a 1000 games. 

I will pass your and your opponent's function into the run alternating games function for maybe 1000 games to see who won more games. That person will be the winner of the match. I think it's a reliable method to compare policies. Run them by each for 1000s of games and see what policy wins the most games.

You have to solve this part using **Q Learning**

In [71]:
import numpy as np

class QLearningAgent:
    def __init__(self, sign ,gamma=0.3, alpha=0.95):
        self.sign = sign
        self.gamma = gamma # also called learning rate
        self.alpha = alpha # also called discount factor
        self.Q_dict = {}

    def create_key(self, state):
        return "".join(state)
    def get_q_values(self, state):
        s_key = self.create_key(state)
        if s_key not in self.Q_dict:
            self.Q_dict[s_key] = np.ones(16).astype(np.float16) * 0.5
        return self.Q_dict[s_key]

    def choose_action(self, state, available_actions):
        q_values = self.get_q_values(state)
        illegal_state = np.array([i for i in range(16) if i not in available_actions])
        if len(illegal_state):
            q_values[illegal_state] = -1.0
        legal_q_vals = q_values[available_actions]
        
        max_q = legal_q_vals.max()
        idxs = np.where(legal_q_vals == max_q)[0]
        return available_actions[np.random.choice(idxs)]

    def update_q_values(self, state_action_history: list, reward):
        
        # updating the reward for the last state
        state, action = state_action_history[-1]
        s_key = self.create_key(state)
        self.Q_dict[s_key][action] = reward

        reward = self.get_q_values(state).max()

        # iterating from last state and action to the first
        # updating the Q matrix based on discount factor
        for state, action in reversed(state_action_history[:-1]):
            s_key = self.create_key(state)
            self.Q_dict[s_key][action] = ( (1 - self.gamma) * self.Q_dict[s_key][action]) + (self.gamma*self.alpha*reward)
            reward = self.get_q_values(state).max()
        

In [85]:
from tqdm import tqdm
# Initialize the agent
player1 = QLearningAgent(-1) # playing as X
player2 = QLearningAgent(1) # playing as O
exploration_ratio = 0.8
tictactoe = TicTacToe()

# Simulate a number of games
for i in tqdm(range(1000000)):
    # Initialize the game and the state
    tictactoe = TicTacToe()
    board = np.array([" "]*16)
    # to keep track of history to make the update
    state_action_history1 = [] # for player 1
    state_action_history2 = [] # for player 2
    while True:
        # The agent chooses an action
        available_actions = np.where(np.array(board) == " ")[0]
        
        if np.random.rand() < exploration_ratio:
            action = np.random.choice(available_actions)
            player1.get_q_values(board)
        else:
            action = player1.choose_action(board, available_actions)

        # The agent performs the action and gets the reward
        state_action_history1.append([board, action])
        board, player, terminated, reward = tictactoe.step(action)
        board = np.array(board).flatten()
        
        if not terminated:
            # The agent updates the Q-value of the previous state-action pair
            next_available_actions =  np.where(np.array(board) == " ")[0]
            if np.random.rand() < exploration_ratio:
                action = np.random.choice(next_available_actions)
                player2.get_q_values(board)
            else:
                action = player2.choose_action(board, next_available_actions)

            # The agent performs the action and gets the reward
            state_action_history2.append([board, action])
            board, player, terminated, reward = tictactoe.step(action)
            board = np.array(board).flatten()
            
        if terminated:
            break
    
    # When we reach here we will have final reward
    # -1 = O won, player will be 1
    # 1 = X won, player will be -1
    #if reward != 0:
    #    print(reward)
    player1.update_q_values(state_action_history=state_action_history1, reward=reward)
    player2.update_q_values(state_action_history=state_action_history2, reward=reward*-1)

100%|██████████| 1000000/1000000 [30:57<00:00, 538.43it/s]


In [73]:
import pickle as pkl
Q_strategies = {
    "X_strategy": player1.Q_dict,
    "O_strategy": player2.Q_dict
}
with open("Q_strategys.pkl", "wb") as f:
    pkl.dump(Q_strategies, f)

In [87]:
len(player2.Q_dict)

2178986

In [51]:
np.array(board) == " "

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [82]:
board.tolist().index("X")

2

In [37]:
{tuple([1,2]): 1}

{(1, 2): 1}

In [56]:
player2.Q_dict

{'             X  ': array([5.236e-04, 8.941e-06, 2.319e-03, 1.174e-05, 4.356e-04, 3.761e-05,
        3.695e-06, 5.114e-05, 1.303e-03, 0.000e+00, 7.260e-05, 0.000e+00,
        2.475e-04, 0.000e+00, 0.000e+00, 3.848e-04], dtype=float16),
 'O         X  X  ': array([0.       , 0.       , 0.       , 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.0002475, 0.       , 0.       , 0.       ,
        0.       , 0.       , 0.       , 0.       ], dtype=float16),
 'O        OX  X X': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float16),
 'O    X  OOX  X X': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float16),
 'O    X  OOX XXOX': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float16),
 'O XO X  OOX XXOX': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float16),
 'OXXO X OOOX XXOX': array([0., 0., 0., 0., 0., 0., 0., 0., 0.

In [85]:
board.count_

AttributeError: 'numpy.ndarray' object has no attribute 'count'

In [88]:
sum(board == "X") - sum(board == "O")

0

In [89]:
board

array(['O', 'O', 'X', 'X', 'O', 'O', 'O', 'X', 'X', 'X', 'O', 'O', 'X',
       'X', 'O', 'X'], dtype='<U1')